In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore') # Ignore warnings when running the code

# Chargement des données
opportunities = pd.read_csv('../data_sources/raw/opportunity_test.csv')
propositions = pd.read_csv('../data_sources/raw/propositions_test.csv')
banques = pd.read_csv('../data_sources/raw/banques_test.csv')

In [5]:
# Exploration rapide des fichiers banques et propositions
print("=== Analyse du fichier banques ===")
print("\nStructure du fichier banques :")
print(banques.info())
print("\nAperçu des premières lignes :")
print(banques.head())
print("\nValeurs manquantes :")
print(banques.isnull().sum())


=== Analyse du fichier banques ===

Structure du fichier banques :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2881 entries, 0 to 2880
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id           2881 non-null   object
 1   CreatedDate  2881 non-null   object
 2   Name         2881 non-null   object
dtypes: object(3)
memory usage: 67.7+ KB
None

Aperçu des premières lignes :
                   Id          CreatedDate                           Name
0  0011i000002MVpNAAW  2018-10-05 10:50:32                          CFCAL
1  0011i000002MVu7AAG  2018-10-05 10:59:31                          CFCAL
2  0011i000002MWCHAA4  2018-10-05 11:24:29                          CFCAL
3  0011i000003PFlRAAW  2018-10-29 15:18:49  CREDIT DU NORD NORD DE FRANCE
4  0011i000003PFtzAAG  2018-10-29 15:30:39  CREDIT DU NORD NORD DE FRANCE

Valeurs manquantes :
Id             0
CreatedDate    0
Name           0
dtype: int64


In [6]:
print("\n=== Analyse du fichier propositions ===")
print("\nStructure du fichier propositions :")
print(propositions.info())
print("\nStatistiques descriptives des colonnes numériques :")
print(propositions[['TXHA__c', 'DureePret_Mois__c', 'TauxAss__c']].describe())
print("\nValeurs manquantes :")
print(propositions.isnull().sum())



=== Analyse du fichier propositions ===

Structure du fichier propositions :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53458 entries, 0 to 53457
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Id                 53458 non-null  object 
 1   CreatedDate        53458 non-null  object 
 2   Opportunity__c     53458 non-null  object 
 3   Partenaire__c      52891 non-null  object 
 4   TXHA__c            53243 non-null  float64
 5   DureePret_Mois__c  53269 non-null  float64
 6   TauxAss__c         53263 non-null  float64
 7   Etape_Source__c    53458 non-null  object 
dtypes: float64(3), object(5)
memory usage: 3.3+ MB
None

Statistiques descriptives des colonnes numériques :
            TXHA__c  DureePret_Mois__c    TauxAss__c
count  53243.000000       53269.000000  53263.000000
mean       3.968626         265.802305      0.249580
std        1.548539          40.776199      0.301356
min        0

In [7]:

# Vérification des valeurs potentiellement aberrantes
print("\nValeurs extrêmes dans les propositions :")
print("\nTaux d'intérêt (TXHA__c) min/max :")
print(propositions['TXHA__c'].agg(['min', 'max']))
print("\nDurée de prêt (mois) min/max :")
print(propositions['DureePret_Mois__c'].agg(['min', 'max']))
print("\nTaux d'assurance min/max :")
print(propositions['TauxAss__c'].agg(['min', 'max']))


Valeurs extrêmes dans les propositions :

Taux d'intérêt (TXHA__c) min/max :
min      0.0
max    100.0
Name: TXHA__c, dtype: float64

Durée de prêt (mois) min/max :
min      12.0
max    3000.0
Name: DureePret_Mois__c, dtype: float64

Taux d'assurance min/max :
min     0.0
max    50.0
Name: TauxAss__c, dtype: float64


In [ ]:
import os

# Création du dossier gold s'il n'existe pas
cleaned_data_path = '../data_sources/gold'
if not os.path.exists(cleaned_data_path):
    os.makedirs(cleaned_data_path)

# 1. Nettoyage des opportunités
opportunities_clean = opportunities.copy()
age_mask = (opportunities_clean['Age_emprunteur__c'] >= 18) & \
           (opportunities_clean['Age_emprunteur__c'] <= 100)
revenue_mask = (opportunities_clean['TotRev__c'] > 0) & \
               (opportunities_clean['TotRev__c'] <= 20000)
opportunities_clean = opportunities_clean[age_mask & revenue_mask]

# 2. Nettoyage des propositions
propositions_clean = propositions.copy()
proposition_masks = (
    (propositions_clean['TXHA__c'] >= 0.1) & 
    (propositions_clean['TXHA__c'] <= 15) &
    (propositions_clean['DureePret_Mois__c'] >= 12) & 
    (propositions_clean['DureePret_Mois__c'] <= 360) &
    (propositions_clean['TauxAss__c'] >= 0) & 
    (propositions_clean['TauxAss__c'] <= 1) &
    propositions_clean['Partenaire__c'].notna()
)
propositions_clean = propositions_clean[proposition_masks]

# 3. Nettoyage des banques
banques_clean = banques.drop_duplicates(subset=['Name'], keep='first')

# 4. Maintien de la cohérence entre les fichiers
propositions_clean = propositions_clean[
    propositions_clean['Opportunity__c'].isin(opportunities_clean['Id']) &
    propositions_clean['Partenaire__c'].isin(banques_clean['Id'])
]

# Export des fichiers nettoyés
opportunities_clean.to_csv(f'{cleaned_data_path}/opportunities_cleaned.csv', index=False)
propositions_clean.to_csv(f'{cleaned_data_path}/propositions_cleaned.csv', index=False)
banques_clean.to_csv(f'{cleaned_data_path}/banques_cleaned.csv', index=False)

print("Nettoyage terminé ! Les fichiers ont été exportés dans :", cleaned_data_path)

Nettoyage terminé ! Les fichiers ont été exportés dans : data_sources/gold
